# Diffusion Model: Task 3

This notebook includes work performed on task 3 of the project to model diffusion from the inner stream into the bubble.

In task 2, I developed a numerical model of diffusion of carbon dioxide from a bulk fluid reservoir of infinite extent into a spherical bubble with a concentration diffusivity of carbon dioxide through the fluid and estimated the uncertainty in the model due to uncertainty in experimental measurements of the diffusivity as a function of concentration with G-ADSA at the Di Maio lab in 2019. Now, I will relax the second of the two assumptions of the Epstein-Plesset model,

1. a constant diffusivity of carbon dioxide in polyol with respect to carbon dioxide concentration.
2. **a constant carbon dioxide concentration in an infinite bulk.**

Here, we set an initial condition with the saturation concentration of CO2 in the inner stream and no CO2 in the outer stream at the inlet. We allow the CO2 to diffuse into the outer stream as it flows down the channel until reaching the desired nucleation time. At that point, we simultaneously grow the bubble based on the concentration gradient calculated at the bubble surface.

**Challenges**:
1. How do we shift the concentration profile in the fluid as the bubble grows? Should the bubble “consume” the fluid? Should the concentration profile be shifted outward? Should the concentration profile be “squeezed” in between the bubble and the capillary wall?
2. How do we account for different concentration profiles in the axial and radial directions?
3. How do we account for elongation of the bubble?

**Goals**: 

1. Demonstrate agreement of concentration-dependent model developed with the concentration-independent model from task 1 when the model for concentration dependence of diffusivity is constant.
2. Estimate the uncertainty in the diffusivity as a function of concentration of carbon dioxide due to experimental uncertainty 
3. Speed up computation time

## Setup

### Import Libraries

In [1]:
# adds custom libraries to path
import sys
sys.path.append('src/')
sys.path.append('../libs/')

# imports standard libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# imports custom libraries
import polyco2
import diffn
import flow
import bubble
import bubbleflow
import analytics as an
import finitediff as fd

# plotting
import plot.diffn as pltd
import plot.bubble as pltb
import plot.genl as pltg

# CONVERSIONS
from conversions import *
# CONSTANTS
from constants import *

from importlib import reload

### Declare Model Parameters

In [5]:
# flow parameters for experiments in 20200905_80bar folder
# viscosity of inner and outer streams [Pa.s]
eta_i = 0.25 # rough estimate of VORANOL 360 + CO2 dissolved so should be lower
eta_o = 4.815
# length of observation capillary [m]
L = 10E-2
# outer stream radius [m]
R_o = 150E-6
Q_o = 200 # outer stream flow rate, input to ISCO 260 D [uL/min]
# inner stream flow rate [uL/min]
Q_i = 100
# saturation pressure [Pa]
p_s = 80E5
# distance down capillary at which measurements were taken in videos of effect of flow rate on 20200905_80bar [m]
d = 0.096

# bubble growth model parameters
dt = 1E-12 # [s]
R_nuc = 5E-6 # [m] arbitrary bubble size for comparing models

# grid parameters
R_max = 150E-6 # outer radius [m]

# diffusion model parameters 
dc_c_s_frac = 0.01 # step size in concentration for estimating dD/dc as a fraction of saturation concentration

# load data (interfacial tension, solubility, and diffusivity) for polyols? we don't have data for VORANOL 360...use 1k3f
polyol_data_file = 'input/1k3f_22c.csv'

# equation of state data
eos_co2_file = 'input/eos_co2_22-0C.csv'

# plot parameters
t_fs = 18
ax_fs = 16
tk_fs = 14
l_fs = 12

### Calculate Flow Parameters

In [3]:
# computes inner stream radius [m] and velocity [m/s]
dp, R_i, v = flow.get_dp_R_i_v_max(eta_i, eta_o, L, Q_i*uLmin_2_m3s, Q_o*uLmin_2_m3s, R_o, SI=True)
# inlet pressure [Pa]
p_in = P_ATM - dp
# nucleation time [s]
t_nuc = d/v

### Compute Epstein-Plesset Result as a Benchmark

In [4]:
# collects relevant parameters
eps_params = (dt, p_s, R_nuc, L, p_in, v, polyol_data_file, eos_co2_file)

# Epstein-Plesset solution for comparison
t_eps, m, D, p, p_bub, if_tension, c_s, c_bulk, R_eps, rho_co2 = bubble.grow(t_nuc, *eps_params)
props_list_eps = (R_eps, m, p, p_bub, rho_co2, if_tension)

# uses 2nd-order Taylor stencil to compute concentration gradient at the surface of the bubble
dcdr_eps = bubble.calc_dcdr_eps(c_bulk, c_s, R_eps, D, np.asarray(t_eps) - t_nuc)

C:\Users\andyl\OneDrive - California Institute of Technology\Documents\Research\Kornfield\ANALYSIS\bubble-growth\bubble.py:204: RuntimeWarning: divide by zero encountered in true_divide
  1/np.sqrt(np.pi*D*(np.asarray(t))))


### Load Bubble Data

Loads data measured from a bubble recorded in the video `v360_co2_c5_11184...mp4` from October 10, 2020, analyzed in `20201010_v360_co2_c5_11184...ipynb`.

In [ ]:
# frame rate [fps]
fps = 11184

# initial and final frames with bubbles (inclusive)
f_i = 9510
f_f = 9527
R_bubble = 8.5E-6 # bubble radius in first frame [m]
x_bubble = d # distance traveled along channel [m]
t_bubble = x_bubble / v # time bubble had traveled down channel [s]

# additional bubbles
f_bubbles = [i for i in range(f_i+1, f_f+1)]
R_bubbles = np.asarray([8.8, 8.8, 9, 9.8, 9.65, 10, 9.65, 10.15, 10.3, 10.3, 10.5, 11.15, 11.25, 
                      11.8, 11.55, 12.15, 12.8])/m_2_um
t_bubbles = np.asarray([t_bubble + (f - f_i)/fps for f in f_bubbles])

## Analysis

### Incompressible Outer Stream, Expanding Capillary

In this first version, we assume that the fluid around the bubble is incompressible and shifts outward as the bubble grows, and is accommodated by an expanding capillary inner wall.

In [10]:
reload(bubbleflow)
# simulation parameters
R_i = 40E-6
N0 = 50
D_max = diffn.D_p(500) # [m^2/s]
dt_sheath = 0.5*(R_max/N0)**2/D_max
half_grid = True
D_fn = diffn.D_p

# performs simulation
t_flow, c, t_num, m, D, p, p_bub, if_tension, \
c_bub, c_bulk, R, rho_co2, v, dr_list = bubbleflow.sheath_incompressible(t_nuc, eps_params, R_max, N0, dc_c_s_frac, R_i,
                                                                         dt_sheath, half_grid=half_grid, D_fn=D_fn)

0% complete, t = 0.000 ms.
10% complete, t = 30.868 ms.
20% complete, t = 59.362 ms.
30% complete, t = 87.856 ms.
40% complete, t = 118.724 ms.
50% complete, t = 147.217 ms.
60% complete, t = 175.711 ms.
70% complete, t = 204.205 ms.
80% complete, t = 235.073 ms.
90% complete, t = 263.567 ms.
halving grid
halving grid


KeyboardInterrupt: 

#### Plot Results

In [ ]:
r_arr_list = [diffn.make_r_arr_lin(dr, R_max) for dr in dr_list]
n_plot = 50
ax1, ax2 = plot_bubble_sheath(R, m, p, p_bub, rho_co2, if_tension, t_num, t_nuc, t_flow, 
                              r_arr_list, c, R_max, v, c_bulk, n_plot, R_i=R_i)

### Fit to Bubble Growth

Now that I have coupled the inner and outer solutions, I will try to fit the result to our bubble growth profile.

In [ ]:
# fits results to bubble growth model
growth_fn = bubbleflow.sheath_incompressible
i_t = 2 # t_bub to match R
i_R = 10
sigma_R = 0.03 # tolerance of error in radius
# groups arguments for growth model
N = 200
D_max = 3E-9 # [m^2/s]
dt_sheath = 0.5*(R_max/N)**2/D_max
args = [eps_params, R_max, N, dc_c_s_frac, R_i, dt_sheath, D_p]
i_t_nuc = 0

# increases maximum iterations
max_iter = 15

# bounds on nucleation time
t_nuc_lo = 0.26 # [s]
t_nuc_hi = 0.28 # [s]

# opens figure to show results of different guesses for bubble nucleation time
fig = plt.figure()
ax = fig.add_subplot(111)

# uses modified shooting method to estimate the nucleation time
t_nuc, output = an.fit_growth_to_pt(t_bubble, R_bubble, t_nuc_lo, t_nuc_hi, growth_fn, args,
                     i_t_nuc, sigma_R=sigma_R, ax=ax, max_iter=max_iter, i_t=i_t, i_R=i_R)

# unpacks output
t_flow, c, t_bub, m, D, p, p_bub, if_tension, c_bub, c_bulk, R, \
            rho_co2, v, dr_list = output
# groups results for plotting
props_list_fit = (R, m, p, p_bub, rho_co2, if_tension)

#### Plots Results

In [ ]:
reload(pltb)
# log time axis
x_lim = [0.00000005, 4]
y_lim = [0.001, 300]
x_log = True
title = 'Linear model for D(c) (pressurization curve)'
ax = pltb.all_props(t_bub, t_nuc, props_list_fit, x_log=x_log, x_lim=x_lim, y_lim=y_lim, title=title)
ax = pltb.measured(ax, t_nuc, t_bubble, t_bubbles, R_bubble, R_bubbles, t_R=(t_bub, props_list_fit[0]))
ax = pltb.d_infl(ax, t_bub, t_nuc, props_list_fit, c_s[0])
pltg.legend(ax)

# linear time axis
x_lim = [0.0, 4]
y_lim = [1, 50]
x_log = False
ax = pltb.all_props(t_bub, t_nuc, props_list_fit, x_log=x_log, x_lim=x_lim, y_lim=y_lim, title=title)
ax = pltb.measured(ax, t_nuc, t_bubble, t_bubbles, R_bubble, R_bubbles)
ax = pltb.d_infl(ax, t_bub, t_nuc, props_list_fit, c_s[0])

pltg.legend(ax)